# Using For-EachObject -Parallel in PowerShell 7

If you are a PowerShell user you are likely already familiar with the For-EachObject command.
In the past you have used it with the -Proccess parameter.

In [ ]:
1..5 | ForEach-Object -Process {write-output "This is number $_"; sleep 1}

In [ ]:
(Measure-Command { 
    1..5 | ForEach-Object -Process {write-output "This is number $_"; sleep 1}
    }).Seconds

However, with the introduction of the -Parallel paramter in PowerShell 7 we can improve the performance of this command.

In [ ]:
(Measure-Command { 
    1..5 | ForEach-Object -Parallel {write-output "This is number $_"; sleep 1}
    }).Seconds

While this may seem trivial in the example above, you can begin applying this to larger tasks. 

In [ ]:
function test-logWin2 {
    $logs = @(
                'System',
                'Application',
                'PowerShellCore/Operational',
                'Windows PowerShell',
                'Microsoft-Windows-PowerShell/Operational'
    ) 
    Measure-Command -Expression {
        $Logs | foreach-object -throttlelimit 3 -parallel { 
            get-winevent -logname $_ | add-content -path "c:\dump\Log_$($_.Replace('/','-')).log";          
        } 
    } | Select-Object -Property Minutes, Seconds
}
test-logWin2